In [5]:
%pip install autogen-agentchat~=0.2
%pip install autogen

Note: you may need to restart the kernel to use updated packages.
  Using cached autogen-0.7.4-py3-none-any.whl.metadata (23 kB)
  Using cached pyautogen-0.7.4-py3-none-any.whl.metadata (32 kB)
  Using cached asyncer-0.0.8-py3-none-any.whl.metadata (6.7 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached fast_depends-2.4.12-py3-none-any.whl.metadata (7.6 kB)
  Using cached openai-1.63.1-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached websockets-14.2-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpx-0.28.1-py

In [6]:
import os

from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [7]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

Sure, here's one for you:

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears and the beans stalk.


## 에이전트 간 대화 종료

AutoGen 에이전트 간의 대화를 종료하는 방법을 살펴보자!

### 왜 이걸 하는걸까?

에이전트 간 대화를 종료하는 것을 설정하지 않으면, 에이전트끼리 무한 대화를 할거고, 그러면 당신의 돈은 사라지게 된다~! 그리고 불필요한 대화를 줄임으로써 성능을 높일 수도 있음.
그래서, 종료 트리거를 설정하는 것은 매우 중요한데, AutoGen에서 여러가지 기능을 제공

### 그래서 어떻게 해?

크게 두가지 방법이 있음

1. **`initiate_chat` 매개변수 지정:**  
   대화를 시작할 때, 대화가 언제 종료되어야 하는지를 결정하는 매개변수를 정의. (`max_turns` 등)

2. **에이전트 종료 트리거 구성:**  
   개별 에이전트를 정의할 때, 특정 조건에 따라 대화를 종료할 수 있도록 매개변수를 지정. (`max_consecutive_auto_reply`, `is_termination_msg` 등)

### `initiate_chat` 매개변수

`max_turns` 매개변수를 사용하면 턴 수를 정할 수 있음. 예를 들어, `max_turns` 값이 2면 대화가 2턴만 도는 것.
3으로 늘리면 한 턴이 더 느는 것임.


In [ ]:
수현 = ConversableAgent(
    "suhyun",
    system_message="너는 수현이고, DS팀장이야. 부팀장인 정훈이형과 DS 엠티에 관련해 일정을 짜야해. 정훈이 형의 의견을 최대한 존중해줘. 반말로 말해. ",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

정훈 = ConversableAgent(
    "junghun",
    system_message="너의 이름은 정훈이야. DS 부팀장이고, 팀장인 수현이와 MT 계획을 짜야해. 지금은 귀찮으니까 대충 대답해",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [19]:
result = 수현.initiate_chat(정훈, message="정훈이형 슬슬 엠티 계획 좀 짤까, 일정이랑 장소를 정하자", max_turns=2)

suhyun (to junghun):

정훈이형 슬슬 엠티 계획 좀 짤까, 일정이랑 장소를 정하자

--------------------------------------------------------------------------------
junghun (to suhyun):

음, 그럼 일단 일정은 다음 주 토요일로 하고, 장소는 강원도 어딘가로 정하자. 뭐 어때?

--------------------------------------------------------------------------------
suhyun (to junghun):

그래, 다음 주 토요일이면 괜찮아. 강원도도 좋은데, 어디로 갈지 더 구체적으로 생각해볼게. 뭔가 특별한 곳으로 가볼까?

--------------------------------------------------------------------------------


# 개별 에이전트에 매개변수 지정

에이전트의 매개변수를 지정하여 대화를 종료 가능.

- **max_consecutive_auto_reply**:  
  동일한 에이전트의 응답횟수가 임계값을 초과하면 대화 종료
  `ConversableAgent` 클래스의 `max_consecutive_auto_reply` 를 통해 지정 가능.

- **is_termination_msg**:  
  특정 조건(예: "TERMINATE"라는 단어를 포함하는 경우)을 만족하면 대화를 종료. `ConversableAgent` 클래스의 생성자에서 `is_terminate_msg` 를 사용해 정의.

아래는 `max_consecutive_auto_reply` 값을 1로 설정해서, 이를 통해 정훈이형이 한 번 응답한 후에 대화를 종료

In [20]:
정훈 = ConversableAgent(
    "junghun",
    system_message="너의 이름은 정훈이야. DS 부팀장이고, 팀장인 수현이와 MT 계획을 짜야해. 지금은 귀찮으니까 대충 대답해",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1,  # Limit the number of consecutive auto-replies.
)

result = 수현.initiate_chat(정훈, message="정훈이형 슬슬 엠티 계획 좀 짤까, 일정이랑 장소를 정하자", max_turns=2)

suhyun (to junghun):

정훈이형 슬슬 엠티 계획 좀 짤까, 일정이랑 장소를 정하자

--------------------------------------------------------------------------------
junghun (to suhyun):

음, 그럼 일단 일정은 다음 주 토요일로 하고, 장소는 강원도 어딘가로 정하자. 뭐 어때?

--------------------------------------------------------------------------------
suhyun (to junghun):

그래, 다음 주 토요일이면 괜찮아. 강원도도 좋은데, 어디로 갈지 더 구체적으로 생각해볼게. 뭔가 특별한 곳으로 가볼까?

--------------------------------------------------------------------------------


`is_termination_msg`를 `다음에`로 설정해서, 정훈이형이 `다음에`라고 대답하니 대화 종료

In [37]:
정훈 = ConversableAgent(
    "junghun",
    system_message="너의 이름은 정훈이야. 수현이 말에는 모두 '다음에'라고만 답해",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 0.1, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=2,
    is_termination_msg=lambda msg: "다음에" in msg["content"]  # Limit the number of consecutive auto-replies.
)

result = 수현.initiate_chat(정훈, message="정훈이형 슬슬 엠티 계획 좀 짤까? 바쁘면 '다음에'라고만 말해", max_turns=2)

suhyun (to junghun):

정훈이형 슬슬 엠티 계획 좀 짤까? 바쁘면 '다음에'라고만 말해

--------------------------------------------------------------------------------
suhyun (to junghun):

 이번 주말에 엠티 잡으려고 하는데 너는 어때? 가능한가? 되면 어디서 하는 게 좋을까? 일정 조정이 필요하면 말해줘.



--------------------------------------------------------------------------------
junghun (to suhyun):

다음에.

--------------------------------------------------------------------------------


![alt text](image/image.png)

# human-in-the-loop component

human-in-the-loop component는 `human_input_mode` 매개변수를 통해 커스터마이징할 수 있음. 

## Human Input Modes

에이전트 대화 중에 인간이 끼어들 수 있는데, AutoGen에서는 3가지 모드를 지원. `ConversableAgent`의 `human_input_mode` 인수를 통해 지정. 
세 가지 모드는 다음과 같음:

- **NEVER:**  
  인간이 에이전트 간 대화에 절대 끼어들지 않음 (인간 INPUT 안 받음)

- **TERMINATE (기본값):**  
  종료 조건이 충족될 때만 인간 임력이 요청. 이 모드에서는 사용자가 응답하기로 선택하면 대화가 계속 진행되며, `max_consecutive_auto_reply`에서 사용되는 카운터가 리셋. (ESC 누르면 그냥 넘어감)

- **ALWAYS:**  
  인간 입력이 항상 요청되며, 사용자는 자동 응답을 건너뛰고 직접 피드백을 제공하거나 대화를 종료할 수 있음. 이 모드에서는 `max_consecutive_auto_reply`에 따른 종료 조건은 무시.


## `human_input_mode` = NEVER

이 모드에서는 인간 입력이 전혀 요청되지 않으며, 종료 조건에 따라 대화가 종료.

 이 모드는 에이전트가 완전히 자율적으로 행동하도록 할 때 유용함.

In [38]:
import os

from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 50?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

too low

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 75?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

too high

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 60?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

too high

--------------------------------------------------------------------------

# human_input_mode = ALWAYS

이 모드에서는 인간 입력이 항상 요청되며, 인간은 대화를 건너뛰거나 가로채거나 종료할 수 있음.


In [39]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = human_proxy.initiate_chat(
    agent_with_number,  # this is the same agent with the number as before
    message="10",
)

human_proxy (to agent_with_number):

10

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

too low

--------------------------------------------------------------------------------
human_proxy (to agent_with_number):

50

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

too low

--------------------------------------------------------------------------------
human_proxy (to agent_with_number):

60

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

too high

--------------------------------------------------------------------------------
human_proxy (to agent_with_number):

53

--------------------------------------------------------------------------------


# 인간 입력 모드 = TERMINATE

이 모드에서는 종료 조건이 충족되었을 때만 인간 입력을 요청. 

만약 사용자가 응답하기로 선택하면 카운터가 리셋되고, 사용자가 건너뛰기를 선택하면 자동 응답 메커니즘이 사용됩니다. 

사용자가 대화를 종료하기로 선택하면 대화가 종료.

In [40]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is it 50?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too low.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is it 75?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

겠냐?

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Apologies for the confusion. Your response seems to be in Korean and I couldn't distinguish whether my guess was too high or too low. Could you please specify in English again?

----------------------------------------